# 81. 畳み込みの数学的定義

**Unit 0.4: 空間の帰納バイアス - CNN & 局所性の科学**

---

## このノートブックの位置づけ

前回は畳み込みの**直感的な理解**を深めました。今回は、その操作を**数学的に厳密に定義**します。数式を理解することで、実装時のバグを防ぎ、パラメータの意味を正確に把握できるようになります。

```
Unit 0.4 学習マップ
==================

Section A: 畳み込みの世界への入口
├── 80. 畳み込みとは何か（直感編）  ✅ 完了
├── 81. 畳み込みの数学的定義  ← 今ここ
├── 82. NumPyスクラッチ実装（基礎編）
├── 83. NumPyスクラッチ実装（高速化編）
├── 84. 古典的フィルタの解剖学
└── 85. カーネルの可視化と解釈
```

## 学習目標

このノートブックを終えると、以下のことができるようになります：

1. **離散畳み込みの数式を読み書きできる**
2. **相関（Correlation）と畳み込み（Convolution）の違いを説明できる**
3. **パディングの種類（Valid, Same, Full）を使い分けられる**
4. **出力サイズを計算する公式を適用できる**
5. **手計算で小さな畳み込みを実行できる**

## 目次

1. [前提知識の確認](#1-前提知識の確認)
2. [離散畳み込みの厳密な定義](#2-離散畳み込みの厳密な定義)
3. [相関 vs 畳み込み](#3-相関-vs-畳み込み)
4. [パディング（境界処理）の数学](#4-パディング境界処理の数学)
5. [ストライド：「飛ばし読み」の数学](#5-ストライド飛ばし読みの数学)
6. [演習問題：手計算で畳み込み](#6-演習問題手計算で畳み込み)

---

## 環境セットアップ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyBboxPatch
from matplotlib.colors import LinearSegmentedColormap
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)
print("環境準備完了")

---

## 1. 前提知識の確認

畳み込みの数学を理解するために、いくつかの基本操作を確認しておきましょう。

### 1.1 アダマール積（要素ごとの積）

行列 $A$ と $B$ の**アダマール積（Hadamard product）** $A \odot B$ は、同じ位置の要素同士を掛け算する操作です。

$$
(A \odot B)_{ij} = A_{ij} \times B_{ij}
$$

In [ ]:
# アダマール積の例

A = np.array([[1, 2], 
              [3, 4]])

B = np.array([[5, 6], 
              [7, 8]])

# アダマール積（要素ごとの積）
hadamard = A * B  # NumPyでは * がアダマール積

# 行列積（比較用）
matrix_product = A @ B  # NumPyでは @ が行列積

print("行列 A:")
print(A)
print("\n行列 B:")
print(B)
print("\nアダマール積 A ⊙ B (要素ごとの積):")
print(hadamard)
print("\n計算過程: [[1×5, 2×6], [3×7, 4×8]] = [[5, 12], [21, 32]]")
print("\n（参考）行列積 A @ B:")
print(matrix_product)
print("\n注意：アダマール積 ≠ 行列積")

### 1.2 総和（Σ）の復習

畳み込みでは、アダマール積の結果を**すべて足し合わせる**操作が必要です。

$$
\sum_{i=0}^{n-1} \sum_{j=0}^{m-1} (A \odot B)_{ij}
$$

これは行列の全要素の合計です。

In [ ]:
# 全要素の合計

print("アダマール積の結果:")
print(hadamard)
print(f"\n全要素の合計: {np.sum(hadamard)}")
print(f"計算過程: 5 + 12 + 21 + 32 = {5+12+21+32}")

print("\n" + "="*50)
print("畳み込みの1ステップ = アダマール積 → 総和")
print("="*50)

---

## 2. 離散畳み込みの厳密な定義

いよいよ畳み込みの数式です。まず1次元から始めて、2次元へ拡張します。

### 2.1 1次元離散畳み込み

信号 $f$ とカーネル $g$ の**離散畳み込み**は次のように定義されます：

$$
(f * g)[n] = \sum_{k=-\infty}^{\infty} f[k] \cdot g[n - k]
$$

ここで重要なのは **$g[n-k]$** の部分です。これはカーネルを**反転（flip）**して適用することを意味します。

#### なぜ反転するのか？

数学的な畳み込みは、もともと「2つの関数の重なり具合」を測る操作として定義されました。一方をスライドさせながら重なりを計算するとき、反転が自然に現れます。

In [ ]:
# 1次元畳み込みの数式を図解

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# サンプル信号とカーネル
signal = np.array([0, 1, 2, 3, 2, 1, 0])
kernel = np.array([1, 2, 1])  # 重み付き平均

# 上段：元の信号とカーネル
ax1 = axes[0]
ax1.stem(range(len(signal)), signal, basefmt=' ', linefmt='b-', markerfmt='bo', label='信号 f[k]')
ax1.set_title('入力信号 f[k]', fontsize=12)
ax1.set_xlabel('k')
ax1.set_ylabel('値')
ax1.set_xlim(-1, 10)
ax1.grid(True, alpha=0.3)
ax1.legend()

# 中段：カーネルの反転
ax2 = axes[1]
kernel_positions = np.array([0, 1, 2])
ax2.stem(kernel_positions, kernel, basefmt=' ', linefmt='r-', markerfmt='ro', label='元のカーネル g[m]')
ax2.stem(kernel_positions + 5, kernel[::-1], basefmt=' ', linefmt='g-', markerfmt='g^', label='反転カーネル g[-m]')
ax2.set_title('カーネルの反転（数学的畳み込みで必要）', fontsize=12)
ax2.set_xlabel('位置')
ax2.set_ylabel('値')
ax2.set_xlim(-1, 10)
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.annotate('', xy=(4, 1.5), xytext=(3, 1.5),
            arrowprops=dict(arrowstyle='->', color='black', lw=2))
ax2.text(3.5, 1.7, '反転', fontsize=11, ha='center')

# 下段：数式の説明
ax3 = axes[2]
ax3.axis('off')

formula_text = r"""
【1次元離散畳み込みの定義式】

$$
(f * g)[n] = \sum_{k} f[k] \cdot g[n - k]
$$

■ 各記号の意味：
  • f[k] : 入力信号の k 番目の値
  • g[n-k] : カーネルを n の位置に置いて反転したもの
  • n : 出力の位置（0, 1, 2, ...）
  • Σ : カーネルの範囲内で足し合わせ

■ 計算手順（位置 n での出力）：
  1. カーネルを反転する
  2. 反転カーネルを位置 n に配置
  3. 信号とカーネルを要素ごとに掛ける
  4. 掛け算の結果をすべて足す → 出力 (f*g)[n]
"""
ax3.text(0.05, 0.5, formula_text, fontsize=11, family='monospace',
         verticalalignment='center', transform=ax3.transAxes,
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

### 2.2 2次元離散畳み込み

画像 $I$ とカーネル $K$（サイズ $m \times n$）の2次元畳み込みは：

$$
(I * K)[i, j] = \sum_{u=0}^{m-1} \sum_{v=0}^{n-1} I[i+u, j+v] \cdot K[m-1-u, n-1-v]
$$

または、カーネルを事前に反転して考えると：

$$
(I * K)[i, j] = \sum_{u=0}^{m-1} \sum_{v=0}^{n-1} I[i+u, j+v] \cdot K'[u, v]
$$

ここで $K'$ は $K$ を上下左右に反転（180度回転）したカーネルです。

In [ ]:
# 2次元畳み込みの数式を図解

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# サンプル画像とカーネル
image = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [4, 3, 2, 1]
])

kernel = np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
])  # Sobel-X風

kernel_flipped = np.flip(np.flip(kernel, 0), 1)  # 180度回転

# 画像
ax1 = axes[0]
ax1.imshow(image, cmap='Blues')
for i in range(4):
    for j in range(4):
        ax1.text(j, i, f'{image[i,j]}', ha='center', va='center', fontsize=12)
ax1.set_title('入力画像 I', fontsize=12)
ax1.set_xticks(range(4))
ax1.set_yticks(range(4))

# 元のカーネル
ax2 = axes[1]
ax2.imshow(kernel, cmap='RdBu', vmin=-2, vmax=2)
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{kernel[i,j]}', ha='center', va='center', fontsize=12)
ax2.set_title('カーネル K（元）', fontsize=12)
ax2.set_xticks(range(3))
ax2.set_yticks(range(3))

# 反転カーネル
ax3 = axes[2]
ax3.imshow(kernel_flipped, cmap='RdBu', vmin=-2, vmax=2)
for i in range(3):
    for j in range(3):
        ax3.text(j, i, f'{kernel_flipped[i,j]}', ha='center', va='center', fontsize=12)
ax3.set_title("カーネル K'（180°回転）", fontsize=12)
ax3.set_xticks(range(3))
ax3.set_yticks(range(3))

# 数式説明
ax4 = axes[3]
ax4.axis('off')
explanation = """
【カーネルの反転】

数学的畳み込みでは
カーネルを180°回転
（上下左右反転）して
適用します。

K[i,j] → K[m-1-i, n-1-j]

例：
┌─────────┐   ┌─────────┐
│ 1  0 -1│   │-1  0  1│
│ 2  0 -2│ → │-2  0  2│
│ 1  0 -1│   │-1  0  1│
└─────────┘   └─────────┘
"""
ax4.text(0.1, 0.5, explanation, fontsize=10, family='monospace',
         verticalalignment='center', transform=ax4.transAxes)

plt.tight_layout()
plt.show()

print("\n重要：深層学習の実装では、実際には『相関』を使うことが多いです（次のセクション）")

---

## 3. 相関 vs 畳み込み

ここで重要な事実を明らかにします：

> **深層学習における「畳み込み」は、数学的には「相関（correlation）」です**

### 3.1 相関（Cross-Correlation）の定義

**相関**は、カーネルを反転**せずに**適用する操作です：

$$
(I \star K)[i, j] = \sum_{u=0}^{m-1} \sum_{v=0}^{n-1} I[i+u, j+v] \cdot K[u, v]
$$

畳み込みとの違いは、$K[u,v]$ が反転していない点だけです。

In [ ]:
# 相関と畳み込みの違いを実演

def true_convolution_2d(image, kernel):
    """数学的に正しい畳み込み（カーネルを反転）"""
    kernel_flipped = np.flip(np.flip(kernel, 0), 1)
    return cross_correlation_2d(image, kernel_flipped)

def cross_correlation_2d(image, kernel):
    """相関（カーネルを反転しない）- 深層学習で使われる"""
    k_h, k_w = kernel.shape
    i_h, i_w = image.shape
    
    # 出力サイズ（validモード）
    o_h = i_h - k_h + 1
    o_w = i_w - k_w + 1
    
    output = np.zeros((o_h, o_w))
    for i in range(o_h):
        for j in range(o_w):
            output[i, j] = np.sum(image[i:i+k_h, j:j+k_w] * kernel)
    
    return output

# 非対称なカーネルで違いを確認
image = np.array([
    [0, 0, 0, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0]
], dtype=float)

# 非対称カーネル（右上が特徴的）
kernel = np.array([
    [0, 0, 1],
    [0, 0, 0],
    [0, 0, 0]
], dtype=float)

# 計算
corr_result = cross_correlation_2d(image, kernel)
conv_result = true_convolution_2d(image, kernel)

# 可視化
fig, axes = plt.subplots(2, 3, figsize=(14, 9))

# 上段：入力とカーネル
ax1 = axes[0, 0]
ax1.imshow(image, cmap='Blues')
for i in range(5):
    for j in range(5):
        ax1.text(j, i, f'{int(image[i,j])}', ha='center', va='center', fontsize=11)
ax1.set_title('入力画像', fontsize=12)

ax2 = axes[0, 1]
ax2.imshow(kernel, cmap='Oranges', vmin=0, vmax=1)
for i in range(3):
    for j in range(3):
        ax2.text(j, i, f'{int(kernel[i,j])}', ha='center', va='center', fontsize=11)
ax2.set_title('カーネル K\n（右上に1）', fontsize=12)
ax2.add_patch(Rectangle((1.5, -0.5), 1, 1, fill=False, edgecolor='red', linewidth=3))

ax3 = axes[0, 2]
kernel_flip = np.flip(np.flip(kernel, 0), 1)
ax3.imshow(kernel_flip, cmap='Oranges', vmin=0, vmax=1)
for i in range(3):
    for j in range(3):
        ax3.text(j, i, f'{int(kernel_flip[i,j])}', ha='center', va='center', fontsize=11)
ax3.set_title("反転カーネル K'\n（左下に1）", fontsize=12)
ax3.add_patch(Rectangle((-0.5, 1.5), 1, 1, fill=False, edgecolor='red', linewidth=3))

# 下段：結果
axes[1, 0].axis('off')
axes[1, 0].text(0.5, 0.5, '相関 vs 畳み込み\nの違いを確認', 
                fontsize=14, ha='center', va='center', transform=axes[1, 0].transAxes)

ax4 = axes[1, 1]
ax4.imshow(corr_result, cmap='Blues', vmin=0, vmax=1)
for i in range(corr_result.shape[0]):
    for j in range(corr_result.shape[1]):
        ax4.text(j, i, f'{int(corr_result[i,j])}', ha='center', va='center', fontsize=11)
ax4.set_title('相関の結果\n（カーネル反転なし）\n← 深層学習で使用', fontsize=11)

ax5 = axes[1, 2]
ax5.imshow(conv_result, cmap='Blues', vmin=0, vmax=1)
for i in range(conv_result.shape[0]):
    for j in range(conv_result.shape[1]):
        ax5.text(j, i, f'{int(conv_result[i,j])}', ha='center', va='center', fontsize=11)
ax5.set_title('畳み込みの結果\n（カーネル反転あり）\n← 数学的に正確', fontsize=11)

plt.tight_layout()
plt.show()

print("\n【解釈】")
print("- カーネルの『1』は右上にある")
print("- 相関：入力の『右上』を見て値を決める → 結果が左上にずれる")
print("- 畳み込み：カーネル反転後『左下』を見る → 結果が右下にずれる")
print("\n- 対称なカーネル（ガウシアンなど）では、相関と畳み込みの結果は同じ！")

### 3.2 なぜ深層学習では「相関」を使うのか

深層学習で相関を使い、それを「畳み込み」と呼ぶ理由：

1. **カーネルは学習される**：どうせ学習で決まるなら、反転の有無は問題にならない
2. **実装がシンプル**：反転操作が不要
3. **歴史的経緯**：信号処理の用語がそのまま使われた
4. **対称カーネルが多い**：ガウシアン、平均など、対称なカーネルでは結果が同じ

In [ ]:
# 対称カーネルの場合：相関と畳み込みが一致

# 対称カーネル（ガウシアン風）
symmetric_kernel = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
], dtype=float) / 16

# 反転しても同じ
flipped = np.flip(np.flip(symmetric_kernel, 0), 1)

print("対称カーネル K:")
print(symmetric_kernel.round(3))
print("\n反転カーネル K':")
print(flipped.round(3))
print("\nK と K' は同一:", np.allclose(symmetric_kernel, flipped))
print("\n→ 対称カーネルでは、相関 = 畳み込み")

### 3.3 用語の整理

| 用語 | 操作 | 使用される文脈 |
|------|------|---------------|
| **畳み込み（Convolution）** | カーネルを反転して適用 | 数学、信号処理 |
| **相関（Cross-Correlation）** | カーネルをそのまま適用 | 画像処理、パターンマッチング |
| **深層学習の「畳み込み」** | 相関（反転なし） | CNN、torch.nn.Conv2d など |

> 💡 **このシリーズでは、深層学習の慣例に従い、「相関」を「畳み込み」と呼びます。**

---

## 4. パディング（境界処理）の数学

畳み込みの出力サイズは、境界の処理方法によって変わります。

### 4.1 Valid, Same, Full パディング

入力サイズを $n$、カーネルサイズを $k$ としたとき：

| モード | パディング量 | 出力サイズ | 特徴 |
|--------|-------------|-----------|------|
| **Valid** | 0 | $n - k + 1$ | 境界を無視、出力が小さくなる |
| **Same** | $(k-1)/2$ | $n$ | 出力サイズを入力と同じに保つ |
| **Full** | $k-1$ | $n + k - 1$ | 全ての重なりを含む、出力が大きくなる |

In [ ]:
# パディングモードの違いを可視化

def visualize_padding_modes():
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 入力画像（5x5）
    input_size = 5
    kernel_size = 3
    
    modes = [
        ('Valid', 0, input_size - kernel_size + 1),
        ('Same', 1, input_size),
        ('Full', 2, input_size + kernel_size - 1)
    ]
    
    for ax, (mode_name, pad, output_size) in zip(axes, modes):
        # 背景（パディング領域）
        total_size = input_size + 2 * pad
        background = np.zeros((total_size, total_size))
        
        # 入力領域
        input_region = np.ones((input_size, input_size)) * 0.7
        if pad > 0:
            background[pad:-pad, pad:-pad] = input_region
        else:
            background = input_region
        
        ax.imshow(background, cmap='Blues', vmin=0, vmax=1, 
                  extent=[-0.5, total_size-0.5, total_size-0.5, -0.5])
        
        # 入力領域の枠
        rect = Rectangle((pad-0.5, pad-0.5), input_size, input_size,
                         fill=False, edgecolor='blue', linewidth=3, linestyle='-')
        ax.add_patch(rect)
        
        # カーネルの動く範囲（最初と最後の位置）
        # 最初の位置
        k_start = Rectangle((-0.5, -0.5), kernel_size, kernel_size,
                           fill=False, edgecolor='red', linewidth=2, linestyle='--')
        ax.add_patch(k_start)
        
        # 最後の位置
        k_end = Rectangle((output_size-1-0.5, output_size-1-0.5), kernel_size, kernel_size,
                         fill=False, edgecolor='orange', linewidth=2, linestyle='--')
        ax.add_patch(k_end)
        
        ax.set_title(f'{mode_name} パディング\n'
                    f'パディング量: {pad}\n'
                    f'出力サイズ: {output_size}×{output_size}', fontsize=11)
        ax.set_xlim(-1, total_size)
        ax.set_ylim(total_size, -1)
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
    
    # 凡例
    axes[0].plot([], [], 'b-', linewidth=3, label='入力領域 (5×5)')
    axes[0].plot([], [], 'r--', linewidth=2, label='カーネル開始位置')
    axes[0].plot([], [], color='orange', linestyle='--', linewidth=2, label='カーネル終了位置')
    axes[0].legend(loc='upper left', fontsize=9)
    
    plt.suptitle(f'パディングモードの比較（入力: 5×5, カーネル: 3×3）', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_padding_modes()

print("\n【出力サイズの公式】")
print("入力: n×n, カーネル: k×k, パディング: p")
print("出力サイズ = n - k + 1 + 2p")
print("\n• Valid (p=0): 5 - 3 + 1 = 3")
print("• Same  (p=1): 5 - 3 + 1 + 2 = 5")
print("• Full  (p=2): 5 - 3 + 1 + 4 = 7")

### 4.2 パディングの種類

境界を埋める値にも選択肢があります。

In [ ]:
# 様々なパディング方法

# サンプル1次元配列
arr = np.array([1, 2, 3, 4, 5])
pad_width = 3

padding_modes = {
    'constant (0)': np.pad(arr, pad_width, mode='constant', constant_values=0),
    'edge (端の値)': np.pad(arr, pad_width, mode='edge'),
    'reflect (鏡像)': np.pad(arr, pad_width, mode='reflect'),
    'wrap (周期)': np.pad(arr, pad_width, mode='wrap'),
}

fig, ax = plt.subplots(figsize=(14, 6))

colors = plt.cm.Set2(np.linspace(0, 1, len(padding_modes)))
y_positions = np.arange(len(padding_modes)) * 2

for (name, padded), y, color in zip(padding_modes.items(), y_positions, colors):
    # パディング領域と元データで色を分ける
    for i, val in enumerate(padded):
        is_padding = i < pad_width or i >= len(padded) - pad_width
        fc = 'lightgray' if is_padding else color
        ec = 'gray' if is_padding else 'black'
        ax.add_patch(Rectangle((i, y), 0.9, 0.9, facecolor=fc, edgecolor=ec, linewidth=2))
        ax.text(i + 0.45, y + 0.45, f'{val}', ha='center', va='center', fontsize=11)
    
    ax.text(-1.5, y + 0.45, name, ha='right', va='center', fontsize=11)

# 元の配列を表示
ax.text(pad_width + 2, -1.5, '元の配列 [1,2,3,4,5]', ha='center', fontsize=12, fontweight='bold')
ax.axvline(x=pad_width - 0.1, color='red', linestyle='--', linewidth=2)
ax.axvline(x=pad_width + len(arr) - 0.1, color='red', linestyle='--', linewidth=2)

ax.set_xlim(-3, len(padded) + 0.5)
ax.set_ylim(-2, len(padding_modes) * 2)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('様々なパディング方法（灰色=パディング領域）', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n【各パディング方法の特徴】")
print("• constant: 固定値（通常0）で埋める。最も一般的")
print("• edge: 端の値を繰り返す。エッジアーティファクトが少ない")
print("• reflect: 鏡像で埋める。連続性が高い")
print("• wrap: 反対側から持ってくる。周期的なデータに適切")

---

## 5. ストライド：「飛ばし読み」の数学

**ストライド（stride）**は、カーネルを何ピクセルずつ動かすかを決めるパラメータです。

### 5.1 ストライドの効果

- ストライド 1：1ピクセルずつ移動（通常）
- ストライド 2：2ピクセルずつ移動（出力サイズが約半分に）
- ストライド $s$：$s$ ピクセルずつ移動

In [ ]:
# ストライドの効果を可視化

def show_stride_effect():
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    input_size = 7
    kernel_size = 3
    
    strides = [1, 2, 3]
    
    for ax, stride in zip(axes, strides):
        # 入力画像
        img = np.zeros((input_size, input_size))
        ax.imshow(img, cmap='Blues', vmin=0, vmax=1,
                 extent=[-0.5, input_size-0.5, input_size-0.5, -0.5])
        
        # グリッド線
        for i in range(input_size + 1):
            ax.axhline(y=i-0.5, color='gray', linewidth=0.5)
            ax.axvline(x=i-0.5, color='gray', linewidth=0.5)
        
        # カーネル位置を表示
        output_size = (input_size - kernel_size) // stride + 1
        colors = plt.cm.Reds(np.linspace(0.3, 0.9, output_size**2))
        
        idx = 0
        for i in range(output_size):
            for j in range(output_size):
                y = i * stride
                x = j * stride
                rect = Rectangle((x-0.4, y-0.4), kernel_size-0.2, kernel_size-0.2,
                                facecolor=colors[idx], alpha=0.5, edgecolor='red', linewidth=2)
                ax.add_patch(rect)
                # 出力位置番号
                ax.text(x + kernel_size/2 - 0.5, y + kernel_size/2 - 0.5, f'{idx+1}',
                       ha='center', va='center', fontsize=9, fontweight='bold')
                idx += 1
        
        ax.set_title(f'ストライド = {stride}\n'
                    f'出力サイズ: {output_size}×{output_size}\n'
                    f'計算回数: {output_size**2}', fontsize=11)
        ax.set_xlim(-0.5, input_size-0.5)
        ax.set_ylim(input_size-0.5, -0.5)
        ax.set_aspect('equal')
    
    plt.suptitle(f'ストライドによる出力サイズの変化（入力: 7×7, カーネル: 3×3）',
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

show_stride_effect()

print("\n【出力サイズの公式（ストライド考慮）】")
print("入力: n×n, カーネル: k×k, パディング: p, ストライド: s")
print("出力サイズ = floor((n - k + 2p) / s) + 1")
print("\n例（n=7, k=3, p=0）:")
print("• s=1: floor((7-3)/1) + 1 = 5")
print("• s=2: floor((7-3)/2) + 1 = 3")
print("• s=3: floor((7-3)/3) + 1 = 2")

### 5.2 出力サイズの一般公式

入力サイズ $n$、カーネルサイズ $k$、パディング $p$、ストライド $s$ のとき：

$$
\text{出力サイズ} = \left\lfloor \frac{n - k + 2p}{s} \right\rfloor + 1
$$

ここで $\lfloor \cdot \rfloor$ は床関数（切り捨て）です。

In [ ]:
# 出力サイズ計算機

def compute_output_size(input_size, kernel_size, padding, stride):
    """畳み込みの出力サイズを計算"""
    return (input_size - kernel_size + 2 * padding) // stride + 1

# インタラクティブな計算表
print("="*60)
print("出力サイズ計算表")
print("="*60)
print(f"公式: floor((n - k + 2p) / s) + 1")
print("="*60)

# 様々なパラメータの組み合わせ
examples = [
    (28, 3, 0, 1, "MNIST画像に3×3カーネル"),
    (28, 3, 1, 1, "同上 + Sameパディング"),
    (28, 3, 1, 2, "同上 + ストライド2"),
    (224, 7, 3, 2, "ImageNet第1層（VGG）"),
    (224, 3, 1, 1, "ImageNet通常の3×3conv"),
    (56, 3, 1, 2, "ダウンサンプリング層"),
]

print(f"{'入力n':>6} {'カーネルk':>10} {'パディングp':>12} {'ストライドs':>12} {'出力':>6}  説明")
print("-"*80)

for n, k, p, s, desc in examples:
    out = compute_output_size(n, k, p, s)
    print(f"{n:>6} {k:>10} {p:>12} {s:>12} {out:>6}  {desc}")

print("\n" + "="*60)
print("よく使われる設定:")
print("• k=3, p=1, s=1 → 出力サイズ維持（Same）")
print("• k=3, p=1, s=2 → 出力サイズ半減（ダウンサンプリング）")
print("• k=1, p=0, s=1 → チャネル変換（1×1 conv）")
print("="*60)

---

## 6. 演習問題：手計算で畳み込み

理解を確認するために、手計算で畳み込みを実行してみましょう。

### 問題 1: 1次元畳み込み

In [ ]:
# 問題1: 1次元畳み込み

print("="*60)
print("問題1: 1次元畳み込み（相関）")
print("="*60)

signal_1d = np.array([1, 2, 3, 4, 5])
kernel_1d = np.array([1, 0, -1])

print(f"\n信号:   {signal_1d}")
print(f"カーネル: {kernel_1d}")
print(f"\nパディング: なし (valid)")
print(f"ストライド: 1")
print(f"\n出力サイズは？ {len(signal_1d)} - {len(kernel_1d)} + 1 = ?")
print(f"\n各位置での出力値を計算してください:")
print("位置0: [1, 2, 3] ⊙ [1, 0, -1] = ?")
print("位置1: [2, 3, 4] ⊙ [1, 0, -1] = ?")
print("位置2: [3, 4, 5] ⊙ [1, 0, -1] = ?")

In [ ]:
# 問題1の解答

print("="*60)
print("問題1: 解答")
print("="*60)

print(f"\n出力サイズ: {len(signal_1d)} - {len(kernel_1d)} + 1 = 3")
print("\n各位置での計算:")

output_1d = []
for i in range(len(signal_1d) - len(kernel_1d) + 1):
    window = signal_1d[i:i+len(kernel_1d)]
    products = window * kernel_1d
    result = np.sum(products)
    output_1d.append(result)
    print(f"位置{i}: {window} ⊙ {kernel_1d} = {products} → 合計 = {result}")

print(f"\n最終出力: {output_1d}")
print("\n解釈: このカーネル[1, 0, -1]は『右と左の差』を計算している")
print("       出力[-2, -2, -2]は、信号が一定の傾きで増加していることを示す")

### 問題 2: 2次元畳み込み

In [ ]:
# 問題2: 2次元畳み込み

print("="*60)
print("問題2: 2次元畳み込み（相関）")
print("="*60)

image_2d = np.array([
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8]
])

kernel_2d = np.array([
    [1, 0],
    [0, -1]
])

print("\n入力画像 (3×3):")
print(image_2d)
print("\nカーネル (2×2):")
print(kernel_2d)
print("\nパディング: なし (valid)")
print("ストライド: 1")
print(f"\n出力サイズ: ({3}-{2}+1) × ({3}-{2}+1) = ?")
print("\n4つの位置での出力を計算してください")

In [ ]:
# 問題2の解答

print("="*60)
print("問題2: 解答")
print("="*60)

print(f"\n出力サイズ: 2×2")
print("\n各位置での計算:")

k_h, k_w = kernel_2d.shape
output_2d = np.zeros((2, 2))

positions = [(0,0), (0,1), (1,0), (1,1)]
for i, j in positions:
    window = image_2d[i:i+k_h, j:j+k_w]
    products = window * kernel_2d
    result = np.sum(products)
    output_2d[i, j] = result
    
    print(f"\n位置({i},{j}):")
    print(f"  窓: {window.flatten()} (形状2×2を展開)")
    print(f"  カーネル: {kernel_2d.flatten()}")
    print(f"  要素積: {products.flatten()}")
    print(f"  合計: {result}")

print("\n最終出力:")
print(output_2d.astype(int))
print("\n解釈: このカーネル[[1,0],[0,-1]]は『左上と右下の差』を計算")
print("       対角線方向の変化（エッジ）を検出している")

### 問題 3: パディングとストライド

In [ ]:
# 問題3: 出力サイズの計算

print("="*60)
print("問題3: 出力サイズの計算")
print("="*60)

problems = [
    {"n": 32, "k": 5, "p": 2, "s": 1, "desc": "5×5カーネル、Sameパディング"},
    {"n": 32, "k": 3, "p": 0, "s": 2, "desc": "3×3カーネル、ストライド2"},
    {"n": 64, "k": 7, "p": 3, "s": 2, "desc": "VGG第1層風"},
    {"n": 14, "k": 3, "p": 1, "s": 1, "desc": "小さい特徴マップ"},
]

print("\n公式: output = floor((n - k + 2p) / s) + 1")
print("\n以下の出力サイズを計算してください:\n")

for i, prob in enumerate(problems, 1):
    print(f"{i}. {prob['desc']}")
    print(f"   入力: {prob['n']}×{prob['n']}, カーネル: {prob['k']}×{prob['k']}, "
          f"パディング: {prob['p']}, ストライド: {prob['s']}")
    print(f"   出力サイズ = ?\n")

In [ ]:
# 問題3の解答

print("="*60)
print("問題3: 解答")
print("="*60)

for i, prob in enumerate(problems, 1):
    n, k, p, s = prob['n'], prob['k'], prob['p'], prob['s']
    out = (n - k + 2*p) // s + 1
    calc = f"({n} - {k} + 2×{p}) / {s} + 1 = ({n - k + 2*p}) / {s} + 1 = {out}"
    
    print(f"\n{i}. {prob['desc']}")
    print(f"   計算: {calc}")
    print(f"   出力サイズ: {out}×{out}")

---

## まとめ

### このノートブックで学んだこと

1. **離散畳み込みの数式**
   - 1次元: $(f * g)[n] = \sum_k f[k] \cdot g[n-k]$
   - 2次元: 同様に2重のΣで表現

2. **相関 vs 畳み込み**
   - 数学的畳み込み：カーネルを反転
   - 相関（深層学習）：カーネルをそのまま使用
   - 学習されるカーネルでは区別不要

3. **パディング**
   - Valid: パディングなし、出力縮小
   - Same: 出力サイズ維持
   - Full: 全重なりを含む

4. **ストライド**
   - カーネルの移動幅
   - ストライド2でダウンサンプリング

5. **出力サイズ公式**
   - $\text{output} = \lfloor (n - k + 2p) / s \rfloor + 1$

---

## 次のステップ

次のノートブック **82. NumPyスクラッチ実装（基礎編）** では：

- 今回学んだ数式を Python コードに変換
- forループによる愚直な実装で計算過程を完全理解
- 境界処理（パディング）の実装
- ストライドの実装

数式を自分の手で実装することで、畳み込みの理解が確実なものになります。